# Spectral Event Analysis Tutorial

This tutorial is a hands-on introduction to using the [SpectralEvents toolbox](https://github.com/jonescompneurolab/SpectralEvents), **a collection of functions designed to help researchers characterize high-amplitude peaks in the spectral representation of continuous neural signals**. Here, we'll load and analyze data that is distributed with the toolbox, as described in [Shin et al. (2017)](https://doi.org/10.7554/eLife.29086).

First, we'll import some dependencies. Note that `seaborn` is not required, but is added to improve plot styling.

In [ ]:
%matplotlib widget

import sys
import os.path as op
from glob import glob

import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

Now let's import the SpectralEvents toolbox module. You might need to modify the path to this module so that your Python interpreter knows where to find it.

In [ ]:
# set path to SpectralEvents if necessary
#sys.path.append('/home/ryan/SpectralEvents')
import spectralevents as se

In [ ]:
# dataset parameters
data_dir = op.join(op.dirname(se.__file__), 'data') 
subj_ids = [str(id) for id in range(1, 10 + 1)]  # subject IDs 1-10

n_subjs = len(subj_ids)  # number of subjects
n_trials = 200           # number of trials per subject
n_times = 600            # number of time samples per trial
samp_freq = 600          # sampling rate (Hz)

In [ ]:
# load data
hit_trials = list()
data = list()
for id_idx, id in enumerate(subj_ids):
    fname = op.join(data_dir, 'prestim_humandetection_600hzMEG_subject' + id + '.mat')
    raw_data = loadmat(fname)
    hit_trials.append(np.nonzero(raw_data['YorN'].squeeze())[0])  # indices for trials where the stimulus was detected
    data.append(raw_data['prestim_raw_yes_no'])  # MEG time series (trials x samples)

Once you've loaded the data, make sure you understand how it's formatted. There
are 10 subjects, each with 100 detected (i.e., *hit*) trials and 100 undetected (i.e., *miss*) trials. How might you go about selecting only the *hit* trials from the 1st subject?

In [ ]:
# investigate data structure: # trials x time for Subj 1
data[0].shape

## Identifying events in single-subject data
Let's find the spectral events across trial conditions in a single subject,
after which we'll expand the analysis to include all 10 subjects.

In [ ]:
subj_data = data[0]

We've loaded the data and must now set the conditions governing Spectral Event detection:
- `freqs`, the frequency values over which you will calculate your **t**ime-**f**requency **r**esponse (TFR)
- `times`, the time values at which your signal was sampled relative to each epoch or trial
- `event_band`, the bounds of the frequency band in which you will look for Spectral Events
- `thresh_FOM`, the factor-of-the-median threshold that will be used to find suprathreshold spectral power in each frequency bin across the spectrogram(s)

The general workflow for event detection follows this progression: **TIMESERIES** -> **TFR** -> **SPECTRAL EVENTS**

In [ ]:
# set parameters
freqs = list(range(1, 60 + 1))   # fequency values (Hz) over which to calculate TFR
times = np.arange(n_times) / samp_freq  # seconds
event_band = [15, 29]  # beta band (Hz)
thresh_FOM = 6.  # factor-of-the-median threshold

**Step 1**: **TIMESERIES** -> **TFR**

In [ ]:
# calculate TFR
tfrs = se.tfr(subj_data, freqs, samp_freq)

**Step 2**: **TFR** -> **SPECTRAL EVENTS**

(Note that our band-of-interest, as guided by the literature and discussed in more detail in a few moments, is the **beta band (15-29 Hz)**.)

In [ ]:
# find spectral events!!
spec_events = se.find_events(tfr=tfrs, times=times, freqs=freqs,
                             event_band=event_band, threshold_FOM=thresh_FOM)

Before we investigate any events that were detected, let's see what the average spectrogram looks like.

In [ ]:
fig = se.plot_avg_spectrogram(tfr=tfrs, times=times, freqs=freqs,
                              event_band=event_band) 

What if we separate between the two experimental conditions, *hit* versus *miss* trials?

In [ ]:
subj_hit_trials = hit_trials[0]
subj_miss_trials = [idx for idx in range(n_trials) if idx not in subj_hit_trials]

vlim = [0, 6.0e-17]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              vlim=vlim)  # note the vlim arguement!

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              vlim=vlim)  # note the vlim arguement!

A few observations:

1. It appears that there is **beta** activity across time and trials.

2. This activity is more pronounced in *miss* trials specficially.

If the high beta power is indeed driven by a beta *rhythm*, we should be
able to observe it in individual trials. Let's see what a few individual
trials look like.

In [ ]:
example_trials = [0, 27, 65]

vlim = [0, 7.5e-17]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              timeseries=subj_data[subj_hit_trials],
                              example_epochs=example_trials,  # note the example_epochs arguement!
                              vlim=vlim)

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              timeseries=subj_data[subj_miss_trials],
                              example_epochs=example_trials,  # note the example_epochs arguement!
                              vlim=vlim)

In [ ]:
hit_spec_events = [trial_events for trial_idx, trial_events in enumerate(spec_events)
                   if trial_idx in subj_hit_trials]

miss_spec_events = [trial_events for trial_idx, trial_events in enumerate(spec_events)
                    if trial_idx in subj_miss_trials]

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_hit_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              spec_events=hit_spec_events,  # note the spec_events argument!
                              timeseries=subj_data[subj_hit_trials],
                              example_epochs=example_trials,
                              show_events=True,  # note the show_events argument!
                              vlim=vlim)

fig = se.plot_avg_spectrogram(tfr=tfrs[subj_miss_trials],
                              times=times,
                              freqs=freqs,
                              event_band=event_band,
                              spec_events=miss_spec_events, # note the spec_events argument!
                              timeseries=subj_data[subj_miss_trials],
                              example_epochs=example_trials,
                              show_events=True,  # note the show_events argument!
                              vlim=vlim)

In [ ]:
fig = se.plot_avg_spectrogram(tfr=tfr, times=times, freqs=freqs,
                              event_band=event_band, spec_events=spec_events,
                              timeseries=subj_data, example_epochs=[43, 6, 99],
                              vlim=None, show_events=True)  # try vlim=[0, 1.0e-17]

In [ ]:
# calculate time-frequency response (TFR)
tfrs = np.zeros((n_subjs, n_trials, len(freqs), n_times))

for subj_idx, subj_data in enumerate(data):

    # calculate TFR using the Morlet wavelet method
    tfr = se.tfr(subj_data, freqs, samp_freq)
    tfrs[subj_idx, :, :, :] = tfr

In [ ]:
# run spectral event analysis per subject
spec_events_all = list()

for subj_idx, subj_data in enumerate(data):
    tfr = tfrs[subj_idx]

    # find local maxima in TFR
    spec_events = se.find_events(tfr=tfr, times=times, freqs=freqs,
                                 event_band=event_band, threshold_FOM=6.)
    spec_events_all.append(spec_events)
